In [1]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import shutil
import warnings
warnings.filterwarnings('ignore')
import string
import time
import json
import datetime

In [2]:
company_names = pd.read_csv('../Company Names.csv')
columns_cik_numbers = ['Company','CIK','NaN']
cik_numbers = pd.read_csv('../CIK.txt',sep = ':',header = None,names = columns_cik_numbers,low_memory = False)
cik_numbers = cik_numbers.drop(columns='NaN')
punctuations = string.punctuation
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
cik_numbers['Company'] = cik_numbers['Company'].apply(lambda x: str(x).upper())

company_names['Company'] = company_names['Company'].apply(lambda x: str(x).translate(str.maketrans('','',punctuations)))
company_names['Company'] = company_names['Company'].apply(lambda x: str(x).upper())

required_data = pd.merge(company_names,cik_numbers,on = 'Company')
required_data['CIK'] = required_data['CIK'].apply(lambda x: str(int(x)))
required_data = required_data.drop_duplicates(subset=['Company'],ignore_index=True)
manual_data = {'ADVANCED VOICE RECOGNITION':'1342936','AMERICAN SECURITY RES CORP':'1085069',
              'AMERICAN SOFTWARE  CL A':'713425','AUTOMATIC DATA PROCESSING':'8670',
              'AVAYA HLDGS CORP':'1418100','BIGCOMMERCE HOLDIN INC':'1626450',
              'BIOKEY INTERNATIONAL INC':'1242388','BLAQCLOUDS INC':'1517990',
              'BSQUARE CORP':'1054721','BUSYBOX COM INC':'1088316',
              'CERIDIAN HCM HOLDING':'1725057','CHANGSHENG INTERNATIONAL GRO':'1451797',
              'CHINA YANYUN YHU NTL ED GRP':'1773086','CLOUDWARD INC':'1045739',
              'COM GUARDCOM INC':'1103759','DUCK CREEK TECHNOL INC':'1160951',
              'FANTASY ACES DAILY FANTASY':'1653115','FLEXSHARES IBOXX 3YR TAR FD':'',
              'FLEXSHARES IBOXX 5YR TAR FD':'','GTY TECHNOLOGY HOLDINGS':'1682325',
              'HRSOFT INC':'917019','INTEGRATED BUSINESS SYS SVC':'1040863',
              'INTELLIGENT SYSTEM CORP':'320340','INTERNATIONAL LEADERS CAP CO':'1470550',
              'ISHARES IBOXX HIGH YLD CP BD':'','ISHARES IBOXX INVST GR CP BD':'',
              'KIWIBOX COM INC':'838796','LIQUI BOX CORP':'216430',
              'LIVE MICROSYSTEMS INC':'915866','MEDALLIANCE INC':'886037',
              'MEDIATECHNICS CORP':'','MONSTER ARTS':'1423746',
              'PALANTIR TECHNOLOG INC':'1321655','PROGRESS SOFTWARE CORP':'876167',
              'QUANTGATE SYSTEMS INC':'','RORINE INTL HOLDING CORP':'1423586',
              'SAILPOINT TECHNO HLDG':'1627857','SKYBOX INTL INC':'909687',
              'SSC TECHNOLOGIES HLDGS INC':'878390','SYNCHRONOSS TECHNOLOGIES':'1131554',
              'TERADATA CORP':'320045','THEGLOBECOM INC':'1066684',
              'VANCORD CAPITAL INC':'1200528','VDOPH INTERNATIONAL INC':'1499355',
              'VERB TECHNOLOGY CO INC':'1566610','VERTICAL COMPUTER SYS INC':'1099509',
              'VMWARE INC CL A':'1124610','WOD RETAIL SOLUTIONS':'704366',
              'ZOOM VIDEO COMUNICATIONS INC':'1585521','ZOOMINFO TECHNOLOGIES':'1794515'}
manual_data = {'Company':list(manual_data.keys()),'CIK':list(manual_data.values())}
manual_data = pd.DataFrame(manual_data)
required_data = pd.concat([required_data,manual_data],ignore_index = True)
required_data

,Company,CIK
0,2U INC,1459417
1,30DC INC,1118974
2,3D PIONEER SYSTEMS INC,1556753
3,8X8 INC,1023731
4,A10 NETWORKS INC,1580808
...,...,...
287,VERTICAL COMPUTER SYS INC,1099509
288,VMWARE INC CL A,1124610
289,WOD RETAIL SOLUTIONS,704366
290,ZOOM VIDEO COMUNICATIONS INC,1585521


In [4]:
os.makedirs('../Stock Performance Graphs urls')

In [7]:

for company,cik in zip(required_data['Company'][50:100],required_data['CIK'][50:100]):

    urls = {}
    i = 1
    while True:
        try:
            driver_10K = webdriver.Chrome("C:\\Users\\91988\\Downloads\\chromedriver_win32\\chromedriver.exe")
            driver_10K.get("https://www.sec.gov/edgar/browse/?CIK={}&owner=exclude".format(cik))
            time.sleep(3)
            driver_10K.find_element(By.XPATH,"//div[@id='filingsStart']/div[2]/div[3]/h5/a").click()

            view_button = driver_10K.find_element(By.XPATH,"//button[@data-group='annualOrQuarterlyReports']")
            driver_10K.implicitly_wait(1)
            ActionChains(driver_10K).move_to_element(view_button).click(view_button).perform()

            form_type = driver_10K.find_element(By.XPATH,"//input[@placeholder='Search table']")
            form_type.send_keys('10-K')
            form_type.send_keys(Keys.ENTER)
        except:
            break


        try:
            form_type = driver_10K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[1]").text.strip()
            reporting_date = driver_10K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[4]").text.strip()
        except:
            break
        if form_type != '10-K':
            i += 1
            continue


        href = driver_10K.find_element(By.XPATH,f"//tbody/tr[{i}]/td[2]/div/a").get_attribute('href')
        driver_10K.get(href)
        try:
            img_href = driver_10K.find_element(By.XPATH,"//*[contains(.,'Stock Performance Graph')]/following::img").get_attribute('src')
            urls[reporting_date] = img_href
        except:
            pass


        i += 1
        driver_10K.close()

    with open(f'../Stock Performance Graphs urls/{company}.json','w') as f:
        json.dump(urls,f)



